# 🔬 Indústria Microeletrônica no Brasil — Análise Temporal

> **TCC (2021)** — Análise de série temporal dos estados brasileiros e municípios de São Paulo sobre a quantidade de funcionários e estabelecimentos referentes ao ramo da Microeletrônica, observando mudanças no período de 2006 a 2019.

**Dados:** RAIS (Relação Anual de Informações Sociais)  
**Métricas analisadas:** Nº de estabelecimentos e Nº de empregados  
**Recortes geográficos:** Estados do Brasil e Municípios de São Paulo

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Diretório dos dados — ajuste conforme necessário
DATA_DIR = Path("data/raw")

# Paleta de cores consistente
COLORS = px.colors.qualitative.Set2

def carregar_dados(arquivo: str, label_linha: str = "UF", remover_total: bool = True) -> pd.DataFrame:
    """
    Carrega e limpa um arquivo Excel de dados da RAIS.
    
    Parâmetros:
        arquivo: Nome do arquivo .xlsx dentro de DATA_DIR
        label_linha: Valor da primeira coluna a ser removido após transpor (ex: 'UF', 'Município-São Paulo')
        remover_total: Se True, remove a coluna 'Total'
    
    Retorna:
        DataFrame com anos como índice e localidades como colunas
    """
    caminho = DATA_DIR / arquivo
    xlsx = pd.ExcelFile(caminho)
    df = pd.read_excel(xlsx, xlsx.sheet_names[0])
    
    # A primeira linha contém os nomes reais das colunas
    df.columns = df.iloc[0].values.tolist()
    
    # Remove linhas de cabeçalho, totais e notas de rodapé
    linhas_remover = [i for i in [0, 1, 26, 28, 29, 30, 31, 32] if i < len(df)]
    df = df.drop(linhas_remover).reset_index(drop=True)
    
    # Remove linhas extras no final (notas de rodapé), se existirem
    df = df.dropna(how="all")
    
    # Transpõe: localidades viram colunas, anos viram linhas
    df = df.transpose()
    df.columns = df.iloc[0].values.tolist()
    df = df.drop(label_linha)
    df = df.sort_index(ascending=True)
    
    if remover_total and "Total" in df.columns:
        df = df.drop("Total", axis=1)
    
    # Converte valores para numérico
    df = df.apply(pd.to_numeric, errors="coerce")
    
    return df


def plotar_serie_temporal(
    df: pd.DataFrame,
    colunas: list[str],
    titulo: str,
    ylabel: str,
    top_n: int | None = None,
) -> go.Figure:
    """
    Cria um gráfico de série temporal interativo com Plotly.
    
    Parâmetros:
        df: DataFrame com anos como índice
        colunas: Lista de colunas (localidades) para plotar
        titulo: Título do gráfico
        ylabel: Label do eixo Y
        top_n: Se fornecido, plota apenas as top_n primeiras colunas
    """
    cols = colunas[:top_n] if top_n else colunas
    
    fig = go.Figure()
    for i, col in enumerate(cols):
        fig.add_trace(go.Scatter(
            x=df.index,
            y=df[col],
            mode="lines+markers",
            name=col,
            marker=dict(size=8),
            line=dict(color=COLORS[i % len(COLORS)], width=2.5),
            hovertemplate=f"<b>{col}</b><br>Ano: %{{x}}<br>{ylabel}: %{{y:,.0f}}<extra></extra>",
        ))
    
    fig.update_layout(
        title=dict(text=titulo, font=dict(size=18)),
        xaxis_title="Ano",
        yaxis_title=ylabel,
        template="plotly_white",
        hovermode="x unified",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.3,
            xanchor="center",
            x=0.5,
        ),
        height=500,
        margin=dict(l=60, r=30, t=60, b=80),
    )
    
    return fig

---
## 1. Estabelecimentos por Estado

Análise da quantidade de estabelecimentos do setor de microeletrônica por unidade federativa do Brasil.

In [ ]:
estab_estados = carregar_dados("Estabelecimentos_estados.xlsx", label_linha="UF")
estab_estados


### Top 3 estados com mais estabelecimentos

In [ ]:
estados = list(estab_estados.columns)

fig = plotar_serie_temporal(
    estab_estados,
    colunas=estados,
    titulo="Top 3 estados com mais estabelecimentos de microeletrônica",
    ylabel="Nº de Estabelecimentos",
    top_n=3,
)
fig.show()


### Todos os estados

In [ ]:
fig = plotar_serie_temporal(
    estab_estados,
    colunas=estados,
    titulo="Estabelecimentos de microeletrônica — Todos os estados",
    ylabel="Nº de Estabelecimentos",
)
fig.show()


---
## 2. Estabelecimentos por Município de São Paulo

Análise detalhada dos municípios paulistas com maior presença de estabelecimentos do setor.

In [ ]:
estab_municipios_sp = carregar_dados(
    "Analise_Estabelecimentos_3rank_municipio_sp.xlsx",
    label_linha="Município-São Paulo",
    remover_total=False,
)
estab_municipios_sp

### Top 3 municípios de SP com mais estabelecimentos

In [ ]:
municipios_sp = list(estab_municipios_sp.columns)

fig = plotar_serie_temporal(
    estab_municipios_sp,
    colunas=municipios_sp,
    titulo="Top 3 municípios de SP com mais estabelecimentos de microeletrônica",
    ylabel="Nº de Estabelecimentos",
    top_n=3,
)
fig.show()


### Todos os municípios de SP

In [ ]:
fig = plotar_serie_temporal(
    estab_municipios_sp,
    colunas=municipios_sp,
    titulo="Estabelecimentos de microeletrônica — Municípios de SP",
    ylabel="Nº de Estabelecimentos",
)
fig.show()

---
## 3. Empregados por Estado

Análise da quantidade de empregados no setor de microeletrônica por unidade federativa.

In [ ]:
empreg_estados = carregar_dados("Analise_Funcionarios_3rank_estados.xlsx", label_linha="UF")
empreg_estados


### Top 3 estados com mais empregados

In [ ]:
estados_empreg = list(empreg_estados.columns)

fig = plotar_serie_temporal(
    empreg_estados,
    colunas=estados_empreg,
    titulo="Top 3 estados com mais empregados na microeletrônica",
    ylabel="Nº de Empregados",  # BUG CORRIGIDO: antes dizia "Estabelecimentos"
    top_n=3,
)
fig.show()

---
## 4. Empregados por Município de São Paulo

Análise da quantidade de empregados no setor nos municípios paulistas.

In [ ]:
empreg_municipios_sp = carregar_dados(
    "Analise_Funcionarios_3rank_municipio_sp.xlsx",
    label_linha="Município-São Paulo",
    remover_total=False,
)
empreg_municipios_sp

In [ ]:
municipios_sp_empreg = list(empreg_municipios_sp.columns)

fig = plotar_serie_temporal(
    empreg_municipios_sp,
    colunas=municipios_sp_empreg,
    titulo="Top 3 municípios de SP com mais empregados na microeletrônica",
    ylabel="Nº de Empregados",  # BUG CORRIGIDO: antes dizia "Estabelecimentos"
    top_n=3,
)
fig.show()

---
## 📌 Próximos Passos

- [ ] Incluir dados pós-2019 (período pandêmico) — fonte: RAIS/CAGED
- [ ] Adicionar mapas geográficos (choropleth por estado)
- [ ] Criar dashboard interativo com Streamlit
- [ ] Calcular taxas de crescimento (CAGR) e tendências
- [ ] Correlacionar nº de estabelecimentos × nº de empregados
- [ ] Contextualizar com eventos do setor (crise dos chips, pandemia)

> Veja o arquivo `MELHORIAS.md` para o plano completo de modernização.